# 🌐 **ViaVisión — Plataforma de Inteligencia de Riesgo Vial**  
### *Análisis integrado de siniestralidad en Calarcá (2021–2025)*  
**Autores:** Elizabeth Garcés · Gabriel Garzón · Jairo Acevedo  

---

## 🚀 **Descripción del Proyecto**
**ViaVisión** es una plataforma de inteligencia territorial que integra datos de  
**accidentalidad**, **puntos críticos** y **parque automotor** para generar  
**perfiles de riesgo vial** por hexágono (H3) y apoyar decisiones basadas en evidencia.  

Este notebook contiene todo el proceso técnico detrás de la plataforma:  
limpieza de datos, análisis exploratorio, modelación del riesgo y construcción del mapa interactivo.

---

## 🎯 **Objetivo General**
Generar perfiles de riesgo vial específicos que permitan **focalizar intervenciones**,  
optimizando recursos en infraestructura, control vehicular y cultura ciudadana.

---

## 🧭 **Qué se logra en este Notebook**
- 🔎 Limpieza y normalización de datasets (ETL)  
- 🧭 Geocodificación territorial con **H3**  
- 📊 Cálculo de indicadores por hexágono  
- ⚠️ Modelo de **Score de Riesgo (0–100)**  
- 📌 Integración con puntos críticos oficiales  
- 🧠 Sistema automático de **recomendaciones por hexágono**  
- 🗺️ Mapa interactivo profesional con Folium  
- 📄 Generación de **PDF del perfil de riesgo**

---

## 📑 **Datos Utilizados**
- **Vehículos matriculados 2020–2022**  
- **Accidentalidad 2021–2025**  
- **Puntos críticos de intervención vial (Oficina TIC)**  
- Scripts ETL + visualizaciones  

---

## 🛠️ **Tecnologías**
- Python (Pandas, H3, Folium, ReportLab)  
- Leaflet (Front)  
- Vite + JavaScript (Plataforma Web)  
- GitHub Pages (Despliegue)

---

## 👥 **Equipo**
- **Elizabeth Garcés** — análisis, UI, visualización  
- **Gabriel Garzón** — ETL, modelamiento, gráficos  
- **Jairo Acevedo** — validación y soporte analítico  

---

## 📄 **Licencia**
MIT — uso abierto con atribución.

---

> ✨ *ViaVisión: Inteligencia para salvar vidas en la vía.*  


#LIBRERIAS

In [66]:
!pip install pandas
!pip install sodapy
!pip install h3ç
!pip install reportlab

import pandas as pd
import io
import numpy as np
import h3
import unicodedata
import ast
import folium
import json
import random
import ast
import plotly.express as px

from google.colab import drive
drive.mount('/content/drive')

from collections import Counter
from datetime import datetime
from sodapy import Socrata
from folium.plugins import MarkerCluster, FeatureGroupSubGroup


from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter
import base64

ERROR: Invalid requirement: 'h3ç': Expected package name at the start of dependency specifier
    h3ç
    ^
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. CARGA DE DATOS


In [67]:
client = Socrata("www.datos.gov.co", None)

In [68]:
#ACCIDENTES DE TRANSITO DESDE MARZO 2017 A DICIEMBRE DE 2022
accidentes_results = client.get("wacd-xkg8", limit=2000)
accidentes_df = pd.DataFrame.from_records(accidentes_results)

In [69]:
#Sectores críticos mortalidad 2022
mortalidad_results = client.get("ybqk-8s42", limit=2000)
mortalidad_df = pd.DataFrame.from_records(mortalidad_results)

In [70]:
#Vehículos matriculados desde enero de 2020 hasta mayo del 2022 por clase, modelo y tipo de vehículo
vehiculos_results = client.get("bj7e-xc9g", limit=18000)
vehiculos_df = pd.DataFrame.from_records(vehiculos_results)

In [71]:
ruta_acc = "/content/drive/MyDrive/BOOTCAMP ANA_DATOS_AVANZADO/CALARCÁ/accidentes_limpio.csv"
ruta_puntos = "/content/drive/MyDrive/BOOTCAMP ANA_DATOS_AVANZADO/CALARCÁ/Puntos_intervencion.csv"

#GRÁFICOS

##Accidentes por mes y dia

In [45]:
# ============================================================
# 1️⃣ CREAR COLUMNA DE FECHA
# ============================================================
df_acc['fecha'] = pd.to_datetime(df_acc[['year','month','day']], errors='coerce')

# ============================================================
# 2️⃣ NORMALIZAR GRAVEDAD
# ============================================================
df_acc['gravedad_norm'] = df_acc['gravedad'].apply(lambda x: str(x).upper().strip())

# ============================================================
# 3️⃣ ACCIDENTES POR MES Y GRAVEDAD
# ============================================================
# Nos aseguramos que month sea entero y válido
df_acc['month'] = pd.to_numeric(df_acc['month'], errors='coerce')
df_acc = df_acc[df_acc['month'].between(1,12)]

acc_por_mes = (
    df_acc.groupby(['month','gravedad_norm'])
          .size()
          .reset_index(name='CANT_ACCIDENTES')
)

fig_mes = px.line(
    acc_por_mes,
    x='month',
    y='CANT_ACCIDENTES',
    color='gravedad_norm',
    markers=True,
    title='Accidentes por Mes según Gravedad',
    labels={'month':'Mes', 'CANT_ACCIDENTES':'Cantidad de Accidentes', 'gravedad_norm':'Gravedad'}
)
fig_mes.update_traces(mode='lines+markers',
                      hovertemplate='Mes: %{x}<br>Accidentes: %{y}')

fig_mes.show()

# ============================================================
# 4️⃣ ACCIDENTES POR DÍA DE LA SEMANA
# ============================================================
dias_semana = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']

df_acc['day_name'] = pd.Categorical(df_acc['day_name'],
                                    categories=dias_semana,
                                    ordered=True)

acc_por_dia = (
    df_acc.groupby(['day_name','gravedad_norm'])
          .size()
          .reset_index(name='CANT_ACCIDENTES')
)

# Aseguramos orden por día
acc_por_dia = acc_por_dia.sort_values('day_name')

fig_dia = px.line(
    acc_por_dia,
    x='day_name',
    y='CANT_ACCIDENTES',
    color='gravedad_norm',
    markers=True,
    title='Accidentes por Día de la Semana según Gravedad',
    labels={'day_name':'Día de la Semana',
            'CANT_ACCIDENTES':'Cantidad de Accidentes',
            'gravedad_norm':'Gravedad'}
)
fig_dia.update_traces(mode='lines+markers',
                      hovertemplate='Día: %{x}<br>Accidentes: %{y}')

fig_dia.show()


/tmp/ipython-input-1434107877.py:48: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



##Accidentes por hora del dia

In [46]:
# Ensure 'hora_int' column exists; if not, create it.
if 'hora_int' not in df_acc.columns:
    def hour_from_time(s):
        try:
            h = int(str(s).split(":")[0])
            return h
        except:
            return None
    df_acc['hora_int'] = df_acc['hora_siniestro'].apply(hour_from_time)

# Filter out rows where 'hora_int' is NaN (e.g., if hora_siniestro was invalid)
df_acc_filtered = df_acc.dropna(subset=['hora_int'])
df_acc_filtered['hora_int'] = df_acc_filtered['hora_int'].astype(int)

# Group by hour and count accidents
acc_por_hora = (
    df_acc_filtered.groupby('hora_int')
    .size()
    .reset_index(name='CANT_ACCIDENTES')
)

# Sort by hour for better visualization
acc_por_hora = acc_por_hora.sort_values('hora_int')

# Create the bar chart
fig_hora = px.bar(
    acc_por_hora,
    x='hora_int',
    y='CANT_ACCIDENTES',
    text='CANT_ACCIDENTES',
    title='Accidentes por Hora del Día',
    labels={
        'hora_int': 'Hora del Día (24h)',
        'CANT_ACCIDENTES': 'Cantidad de Accidentes'
    },
    color_discrete_sequence=px.colors.qualitative.Plotly
)

# Update traces for better text display and hover info
fig_hora.update_traces(
    texttemplate='%{text}',
    textposition='outside',
    hovertemplate='<b>Hora:</b> %{x}:00<br><b>Accidentes:</b> %{y}<extra></extra>'
)

# Customize layout
fig_hora.update_layout(
    xaxis=dict(tickmode='linear', dtick=2),
    yaxis_title='Cantidad de Accidentes',
    showlegend=False
)

fig_hora.show()

## Género por categoría de edad

In [47]:
# Agrupar por categoria_edad y sumar cantidad de hombres y mujeres
acc_por_edad = df_acc.groupby('categoria_edad').agg(
    Hombres=('cantidad_hombres', 'sum'),
    Mujeres=('cantidad_mujeres', 'sum')
).reset_index()

# Transformar a formato "long" para Plotly
acc_por_edad_long = acc_por_edad.melt(id_vars='categoria_edad',
                                      value_vars=['Hombres', 'Mujeres'],
                                      var_name='Genero',
                                      value_name='Cantidad')

# Ordenar categorías de edad si es necesario
edad_orden = sorted(acc_por_edad['categoria_edad'].unique())
acc_por_edad_long['categoria_edad'] = pd.Categorical(acc_por_edad_long['categoria_edad'],
                                                     categories=edad_orden, ordered=True)

# Calcular totales
total_hombres = acc_por_edad['Hombres'].sum()
total_mujeres = acc_por_edad['Mujeres'].sum()
total_personas = total_hombres + total_mujeres
texto_totales = f"Hombres: {total_hombres}  |  Mujeres: {total_mujeres}  |  Total: {total_personas}"

# Gráfico de barras agrupadas
fig_edad = px.bar(
    acc_por_edad_long,
    x='categoria_edad',
    y='Cantidad',
    color='Genero',
    barmode='group',
    title='Cantidad de Hombres y Mujeres por Categoría de Edad',
    labels={'categoria_edad':'Categoría de Edad', 'Cantidad':'Cantidad de Accidentes', 'Genero':'Género'}
)

# Agregar anotación con totales
fig_edad.add_annotation(
    text=texto_totales,
    xref="paper", yref="paper",
    x=0.85, y=1.1,  # posición sobre el gráfico
    showarrow=False,
    font=dict(size=14, color="black"),
    bgcolor="lightyellow",
    bordercolor="black",
    borderwidth=1,
    borderpad=5
)

fig_edad.show()

In [48]:
# Contar accidentes por tipo
acc_por_tipo = (
    df_acc.groupby("tipo_accidente")
          .size()
          .reset_index(name="CANT_ACCIDENTES")
)

# Ordenar de mayor a menor para mejor visualización
acc_por_tipo = acc_por_tipo.sort_values("CANT_ACCIDENTES", ascending=False)

# Crear gráfico de barras
fig_tipo = px.bar(
    acc_por_tipo,
    x="tipo_accidente",
    y="CANT_ACCIDENTES",
    text="CANT_ACCIDENTES",
    title="Cantidad de Accidentes por Tipo de Accidente",
    labels={"tipo_accidente":"Tipo de Accidente", "CANT_ACCIDENTES":"Cantidad de Accidentes"},
    color="tipo_accidente",
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Mostrar los valores encima de las barras
fig_tipo.update_traces(textposition='outside')

# Ajustes de layout
fig_tipo.update_layout(
    xaxis_tickangle=-45,
    yaxis_title="Cantidad de Accidentes",
    showlegend=False,
    uniformtext_minsize=12,
    uniformtext_mode='hide'
)

fig_tipo.show()


#FACTOR OBSOLESENCIA PARQUE AUTOMOTOR

In [72]:
# 1. FUNCIÓN DE NORMALIZACIÓN DE TEXTO
# ============================================================
def normalizar(txt):
    if pd.isna(txt):
        return ""
    return unicodedata.normalize('NFKD', str(txt)).encode('ASCII', 'ignore').decode('utf-8').upper().strip()

try:
    df_acc = pd.read_csv(ruta_acc, encoding="utf-8")
    print("✅ Dataset de accidentes cargado correctamente (CSV).")
except FileNotFoundError:
    print("❌ ERROR: No se encontró el archivo de accidentes.")
    df_acc = pd.DataFrame()
    print("⚠ df_acc quedó vacío.")


# ============================================================
# 3. CARGA PARQUE AUTOMOTOR REAL (vehiculos_df)
# ============================================================
try:
    vehiculos_df = pd.DataFrame.from_records(vehiculos_results)
    print("✅ Parque automotor cargado desde vehiculos_results.")
except:
    print("❌ ERROR: 'vehiculos_results' no existe o tiene formato incorrecto.")
    vehiculos_df = pd.DataFrame()


# ============================================================
# 4. NORMALIZAR CLASE Y MODELO DEL PARQUE AUTOMOTOR
# ============================================================
if not vehiculos_df.empty:

    vehiculos_df["clase_norm"] = vehiculos_df["clase"].apply(normalizar)

    vehiculos_df["modelo"] = pd.to_numeric(vehiculos_df["modelo"], errors="coerce")

    anio_actual = 2025
    vehiculos_df["ANTIGUEDAD"] = anio_actual - vehiculos_df["modelo"]

    vehiculos_df["ES_OBSOLETO"] = vehiculos_df["ANTIGUEDAD"] > 15

    print("✅ Obsolescencia calculada.")
else:
    print("⚠ vehiculos_df está vacío, no se puede calcular obsolescencia.")

# 5. CALCULAR FACTOR DE OBSOLESCENCIA POR CLASE
# ============================================================
if not vehiculos_df.empty:

    riesgo_obsolescencia = (
        vehiculos_df
        .groupby("clase_norm")["ES_OBSOLETO"]
        .mean()
        .reset_index()
    )

    riesgo_obsolescencia.columns = ["TIPO_VEHICULO", "FACTOR_OBSOLESCENCIA"]

    print("\n=== FACTOR DE OBSOLESCENCIA POR TIPO DE VEHÍCULO ===")
    print(riesgo_obsolescencia)

else:
    riesgo_obsolescencia = pd.DataFrame()
    print("⚠ No se pudo generar factor de obsolescencia.")


✅ Dataset de accidentes cargado correctamente (CSV).
✅ Parque automotor cargado desde vehiculos_results.
✅ Obsolescencia calculada.

=== FACTOR DE OBSOLESCENCIA POR TIPO DE VEHÍCULO ===
            TIPO_VEHICULO  FACTOR_OBSOLESCENCIA
0               AUTOMOVIL              0.712484
1                     BUS              0.825688
2                  BUSETA              0.926471
3                  CAMION              0.753939
4               CAMIONETA              0.700811
5                 CAMPERO              0.903689
6              CICLOMOTOR              0.666667
7              CUATRIMOTO              1.000000
8     MAQUINARIA AGRICOLA              1.000000
9   MAQUINARIA INDUSTRIAL              1.000000
10               MICROBUS              0.800000
11            MINITRACTOR              0.000000
12              MOTOCARRO              0.060000
13            MOTOCICLETA              0.530231
14           MOTOTRICICLO              1.000000
15               REMOLQUE              0.25000

In [73]:
#TABLA FACTOR DE OBSOLESENCIA

riesgo_obsolescencia["FACTOR (%)"] = (riesgo_obsolescencia["FACTOR_OBSOLESCENCIA"] * 100).round(2)

# ----- Tabla estilizada -----
tabla_estilizada = (
    riesgo_obsolescencia[["TIPO_VEHICULO", "FACTOR (%)"]]
    .style
    .hide(axis='index')
    .set_caption("Factor de Obsolescencia por Tipo de Vehículo (< 15 años)")
    .set_table_styles([
        {"selector": "caption",
         "props": [("font-size", "18px"),
                   ("font-weight", "bold"),
                   ("margin-bottom", "10px")]},
        {"selector": "th",
         "props": [("background-color", "#1f4e79"),
                   ("color", "white"),
                   ("text-align", "center"),
                   ("padding", "8px")]},
        {"selector": "td",
         "props": [("text-align", "center"),
                   ("padding", "8px")]}
    ])
    .format({"FACTOR (%)": "{:.2f}%"})
    .background_gradient(
        subset=["FACTOR (%)"],
        cmap="YlOrRd"
    )
)

tabla_estilizada

TIPO_VEHICULO,FACTOR (%)
AUTOMOVIL,71.25%
BUS,82.57%
BUSETA,92.65%
CAMION,75.39%
CAMIONETA,70.08%
CAMPERO,90.37%
CICLOMOTOR,66.67%
CUATRIMOTO,100.00%
MAQUINARIA AGRICOLA,100.00%
MAQUINARIA INDUSTRIAL,100.00%


## Diccionario de pesos

In [74]:
# ============================================================
# BLOQUE COMPLETO: TABLAS DE PESOS DEL MODELO DE RIESGO

# -----------------------------
# 1. PESOS POR GRAVEDAD
# -----------------------------
pesos_gravedad = pd.DataFrame({
    "gravedad": [
        "SOLO DAÑOS",
        "HERIDO",
        "MUERTO"
    ],
    "peso_gravedad": [
        1,  # Daños materiales → bajo impacto
        3,  # Herido → impacto medio
        6   # Muerto → impacto máximo
    ]
})

# -----------------------------
# 2. PESOS POR TIPO DE ACCIDENTE
# -----------------------------
pesos_tipo_accidente = pd.DataFrame({
    "tipo_accidente": [
        "CHOQUE",
        "ATROPELLO",
        "VOLCAMIENTO",
        "CAIDA DE OCUPANTE",
        "OTRO"
    ],
    "peso_tipo_accidente": [
        1,   # Choque → más común, menos severo
        3,   # Atropello → severo
        2,   # Volcamiento → medio-alto
        2,   # Caída de ocupante → medio
        1    # Otros → bajo
    ]
})

# -----------------------------
# 3. PESOS POR TIPO DE VEHÍCULO
# -----------------------------
pesos_vehiculo = pd.DataFrame({
    "vehiculo": [
        "MOTO",
        "BICICLETA",
        "AUTOMOVIL",
        "CAMIONETA",
        "CAMION",
        "BUS",
        "TRACTOCAMION",
        "VOLQUETA"
    ],
    "peso_vehiculo": [
        3,    # Motos → altísimo riesgo
        1,    # Bicicletas → riesgo bajo
        1,    # Autos → riesgo bajo
        1.5,  # Camioneta → medio
        2.5,  # Camión → alto
        2,    # Bus → medio-alto
        3.5,  # Tractocamión → muy alto
        3     # Volqueta → alto
    ]
})

# -----------------------------
# 4. PESOS POR ZONA
# -----------------------------
pesos_zona = pd.DataFrame({
    "zona_accidente": [
        "URBANO",
        "RURAL"
    ],
    "peso_zona": [
        1.0,  # Urbano → base
        1.3   # Rural → mayor probabilidad de fatalidad
    ]
})

# -----------------------------
# 5. PESOS POR CANTIDAD DE INVOLUCRADOS
# -----------------------------
pesos_involucrados = pd.DataFrame({
    "rango_involucrados": [
        "1",
        "2",
        "3+"
    ],
    "factor_involucrados": [
        1.0,  # Un solo actor → bajo impacto
        2,  # Dos actores → mayor energía del choque
        3   # Tres o más → escenario grave
    ]
})

# -----------------------------
# 6. DICCIONARIO FINAL
# -----------------------------
pesos_modelo = {
    "gravedad": pesos_gravedad,
    "tipo_accidente": pesos_tipo_accidente,
    "vehiculo": pesos_vehiculo,
    "zona": pesos_zona,
    "involucrados": pesos_involucrados
}

#SCORE INICIAL DE RIESGO POR ACCIDENTE


In [75]:
# ============================================================
# CÁLCULO LIMPIO Y ACTUALIZADO DEL SCORE INDIVIDUAL
# Función para normalizar textos
# ------------------------------------------------------------
def normalizar(txt):
    if pd.isna(txt):
        return ""
    return (
        unicodedata.normalize('NFKD', str(txt))
        .encode('ASCII', 'ignore')
        .decode('utf-8')
        .upper()
        .strip()
    )


# ============================================================
# 1. TABLAS DE PESOS (SE SIGUEN USANDO)
# ============================================================

pesos_gravedad = pd.DataFrame({
    "gravedad": ["SOLO DAÑOS","HERIDO","MUERTO"],
    "peso_gravedad": [1, 3, 6]
})

pesos_tipo_accidente = pd.DataFrame({
    "tipo_accidente": ["CHOQUE","ATROPELLO","VOLCAMIENTO","CAIDA DE OCUPANTE","OTRO"],
    "peso_tipo_accidente": [1,3,2,2,1]
})

pesos_vehiculo = pd.DataFrame({
    "vehiculo": [
        "MOTO","BICICLETA","AUTOMOVIL","CAMIONETA",
        "CAMION","BUS","TRACTOCAMION","VOLQUETA"
    ],
    "peso_vehiculo": [3,1,1,1.5,2.5,2,3.5,3]
})

pesos_zona = pd.DataFrame({
    "zona_accidente": ["URBANO","RURAL"],
    "peso_zona": [1.0, 1.3]
})

pesos_modelo = {
    "gravedad": pesos_gravedad,
    "tipo_accidente": pesos_tipo_accidente,
    "vehiculo": pesos_vehiculo,
    "zona": pesos_zona
}


# ============================================================
# 2. MAPEO SEGURO (NO FALLA SI HAY CATEGORÍAS NUEVAS)
# ============================================================

def map_from_df(value, df, col_key, col_value, default=1):
    v = normalizar(value)
    df_tmp = df.copy()
    df_tmp["norm"] = df_tmp[col_key].apply(normalizar)
    fila = df_tmp[df_tmp["norm"] == v]
    if len(fila) > 0:
        return fila.iloc[0][col_value]
    return default


# ============================================================
# 3. VEHÍCULOS (manejo de combinaciones MOTO-CAMION, etc.)
# ============================================================

def score_vehiculo_tipo(cadena):
    cadena = normalizar(cadena)
    veh = pesos_modelo["vehiculo"].copy()
    veh["norm"] = veh["vehiculo"].apply(normalizar)
    score_max = 1
    for _, row in veh.iterrows():
        if row["norm"] in cadena:
            score_max = max(score_max, row["peso_vehiculo"])
    return score_max


# ============================================================
# 4. FACTOR DE OBSOLESCENCIA
# Requiere la tabla riesgo_obsolescencia ya cargada
# ============================================================

# Diccionario canónico de vehículos
CANONICAS = {
    "MOTO": ["MOTO","MOTOCICLETA"],
    "BICICLETA": ["BICICLETA","BICI"],
    "AUTOMOVIL": ["AUTOMOVIL","AUTO","CARRO"],
    "CAMIONETA": ["CAMIONETA","PICKUP"],
    "CAMION": ["CAMION","CAMION-","TRUCK"],
    "BUS": ["BUS","AUTOBUS"],
    "TRACTOCAMION": ["TRACTOCAMION","TRACTO"],
    "VOLQUETA": ["VOLQUETA"]
}

canon_map = {}
for canon, variantes in CANONICAS.items():
    for v in variantes:
        canon_map[normalizar(v)] = canon

def map_to_canon_simple(text):
    t = normalizar(text)
    if t in canon_map:
        return canon_map[t]
    for k, canon in canon_map.items():
        if k in t:
            return canon
    return None

def extraer_principal(tv):
    tvn = normalizar(tv)
    partes = [p for p in tvn.replace("/", "-").split("-") if p]
    # prioridad
    prioridad = [
        "TRACTOCAMION","VOLQUETA","CAMION","BUS",
        "CAMIONETA","AUTOMOVIL","MOTO","BICICLETA"
    ]
    for p in prioridad:
        if p in partes:
            return p
    return partes[0] if partes else ""


# Crear dict de obsolescencia
if "riesgo_obsolescencia" in globals():
    ro = riesgo_obsolescencia.copy()
    ro["norm"] = ro["TIPO_VEHICULO"].astype(str).apply(normalizar)
    ro["veh_canon"] = ro["norm"].apply(map_to_canon_simple)
    obsol_por_canon = ro.dropna(subset=["veh_canon"]).groupby("veh_canon")["FACTOR_OBSOLESCENCIA"].mean().reset_index()
    dict_obsol = dict(zip(obsol_por_canon["veh_canon"], obsol_por_canon["FACTOR_OBSOLESCENCIA"]))
else:
    dict_obsol = {}

def get_obsol_safe(tv):
    canon_full = map_to_canon_simple(tv)
    if canon_full:
        return dict_obsol.get(canon_full, 0.5)
    principal = extraer_principal(tv)
    canon_prin = map_to_canon_simple(principal)
    if canon_prin:
        return dict_obsol.get(canon_prin, 0.5)
    return 0.5


# ============================================================
# 5. CALCULAR EL SCORE INDIVIDUAL
# ============================================================

df_acc["score_gravedad"] = df_acc["gravedad"].apply(
    lambda x: map_from_df(x, pesos_modelo["gravedad"], "gravedad", "peso_gravedad")
)

df_acc["score_tipo_accidente"] = df_acc["tipo_accidente"].apply(
    lambda x: map_from_df(x, pesos_modelo["tipo_accidente"], "tipo_accidente", "peso_tipo_accidente")
)

df_acc["score_vehiculo"] = df_acc["tipo_vehiculo"].apply(score_vehiculo_tipo)

df_acc["score_zona"] = df_acc["zona_accidente"].apply(
    lambda x: map_from_df(x, pesos_modelo["zona"], "zona_accidente", "peso_zona")
)

df_acc["FACTOR_OBSOLESCENCIA"] = df_acc["tipo_vehiculo"].apply(get_obsol_safe)

# Ajuste por obsolescencia
alpha = 0.25
df_acc["score_vehiculo_adj"] = df_acc["score_vehiculo"] * (1 + alpha * df_acc["FACTOR_OBSOLESCENCIA"])

# Score final
df_acc["SCORE_ACCIDENTE"] = (
    df_acc["score_gravedad"] * 0.35 +
    df_acc["score_tipo_accidente"] * 0.25 +
    df_acc["score_vehiculo_adj"] * 0.25 +
    df_acc["score_zona"] * 0.10 +
    1.0 * 0.05  # involucrados: si quieres agregamos luego
)

print("Cálculo del SCORE_ACCIDENTE completado.")
df_acc[["Id","SCORE_ACCIDENTE"]].head()

Cálculo del SCORE_ACCIDENTE completado.


,Id,SCORE_ACCIDENTE
0,1,2.322134
1,2,2.322134
2,3,1.044530
3,4,1.482119
4,5,2.186326


#PERFIL DE RIESGO POR TIPO VEHICULO

In [76]:
# ============================================================
# PERFIL DE RIESGO POR TIPO DE VEHÍCULO
# ============================================================

# 1) Lista canónica
CANONICAS = {
    "MOTO": ["MOTO", "MOTOCICLETA", "MOTORCYCLE"],
    "BICICLETA": ["BICICLETA", "BICI", "BICYCLE"],
    "AUTOMOVIL": ["AUTOMOVIL", "AUTOMÓVIL", "CARRO", "AUTO"],
    "CAMIONETA": ["CAMIONETA", "PICKUP"],
    "CAMION": ["CAMION", "CAMIÓN", "TRUCK"],
    "BUS": ["BUS", "BUSES", "AUTOBUS"],
    "TRACTOCAMION": ["TRACTOCAMION", "TRACTO", "TRACTOCAMIÓN"],
    "VOLQUETA": ["VOLQUETA"]
}

canon_map = {}
for canon, vars_list in CANONICAS.items():
    for v in vars_list:
        canon_map[normalizar(v)] = canon

def map_to_canon(text):
    t = normalizar(text)
    if t in canon_map:
        return canon_map[t]
    for key, canon in canon_map.items():
        if key in t:
            return canon
    return None


# ------------------------------------------------------------
# 2) Expandir vehículos
# ------------------------------------------------------------
df_acc_exp = df_acc.copy()
df_acc_exp["veh_list"] = df_acc_exp["tipo_vehiculo"].astype(str).apply(
    lambda x: [p for p in normalizar(x).split("-") if p.strip()]
)
df_acc_exp = df_acc_exp.explode("veh_list").reset_index(drop=True)
df_acc_exp["veh_canon"] = df_acc_exp["veh_list"].apply(map_to_canon)
df_acc_exp = df_acc_exp.dropna(subset=["veh_canon", "SCORE_ACCIDENTE"])


# ------------------------------------------------------------
# 3) Severidad promedio por vehículo
# ------------------------------------------------------------
score_accidente_por_vehiculo = (
    df_acc_exp.groupby("veh_canon")
              .agg(
                  NUM_ACCIDENTES=("Id", "count"),
                  SCORE_PROMEDIO_ACC=("SCORE_ACCIDENTE", "mean")
              )
              .reset_index()
)


# ------------------------------------------------------------
# 4) Parque automotor (si no existe → PARQUE=0 y OBSOLESCENCIA=0)
# ------------------------------------------------------------
vehiculos_df = vehiculos_df.copy()
vehiculos_df["veh_canon"] = vehiculos_df["clase"].astype(str).apply(
    lambda x: map_to_canon(normalizar(x))
)

exposicion = (
    vehiculos_df.dropna(subset=["veh_canon"])
               .groupby("veh_canon")
               .size()
               .reset_index(name="PARQUE_TOTAL")
)

# Obsolescencia
if "riesgo_obsolescencia" in globals() and not riesgo_obsolescencia.empty:
    ro = riesgo_obsolescencia.copy()
    ro["veh_canon"] = ro["TIPO_VEHICULO"].astype(str).apply(lambda x: map_to_canon(normalizar(x)))
    obsol_por_canon = ro.dropna(subset=["veh_canon"]).groupby("veh_canon")["FACTOR_OBSOLESCENCIA"].mean().reset_index()
else:
    obsol_por_canon = pd.DataFrame(columns=["veh_canon", "FACTOR_OBSOLESCENCIA"])

# Unir
exposicion = exposicion.merge(obsol_por_canon, on="veh_canon", how="left")

# <<< AJUSTE CRÍTICO >>>
# Si el vehículo NO está en el parque → obsolescencia = 0
exposicion["FACTOR_OBSOLESCENCIA"] = exposicion["FACTOR_OBSOLESCENCIA"].fillna(0)


# ------------------------------------------------------------
# 5) Unir accidentes + exposición
# ------------------------------------------------------------
perfil_vehiculos = score_accidente_por_vehiculo.merge(
    exposicion, on="veh_canon", how="left"
)

perfil_vehiculos["PARQUE_TOTAL"] = perfil_vehiculos["PARQUE_TOTAL"].fillna(0)
perfil_vehiculos["FACTOR_OBSOLESCENCIA"] = perfil_vehiculos["FACTOR_OBSOLESCENCIA"].fillna(0)


# ------------------------------------------------------------
# 6) Normalización
# ------------------------------------------------------------
max_acc = perfil_vehiculos["NUM_ACCIDENTES"].max()

perfil_vehiculos["FREQ_NORM"] = (
    perfil_vehiculos["NUM_ACCIDENTES"] / max_acc if max_acc > 0 else 0
)

max_parque = perfil_vehiculos["PARQUE_TOTAL"].max()

perfil_vehiculos["PARQUE_NORM"] = perfil_vehiculos["PARQUE_TOTAL"].apply(
    lambda x: (x / max_parque) if max_parque > 0 else 0
)

# Si PARQUE_TOTAL = 0 → no cuenta en el score
perfil_vehiculos["PESO_PARQUE"] = perfil_vehiculos["PARQUE_TOTAL"].apply(
    lambda x: 0 if x == 0 else 0.15
)


# ------------------------------------------------------------
# 7) Score final
# ------------------------------------------------------------
factor_obsol = 1 + 0.25 * perfil_vehiculos["FACTOR_OBSOLESCENCIA"]

perfil_vehiculos["SCORE_RAW"] = (
    perfil_vehiculos["SCORE_PROMEDIO_ACC"] * factor_obsol * 0.50 +
    perfil_vehiculos["FREQ_NORM"] * 0.25 +
    perfil_vehiculos["PARQUE_NORM"] * perfil_vehiculos["PESO_PARQUE"] +
    perfil_vehiculos["FACTOR_OBSOLESCENCIA"] * 0.10
)

min_v = perfil_vehiculos["SCORE_RAW"].min()
max_v = perfil_vehiculos["SCORE_RAW"].max()

perfil_vehiculos["SCORE_0_100"] = (
    (perfil_vehiculos["SCORE_RAW"] - min_v) / (max_v - min_v) * 100
    if max_v != min_v else 0
)

perfil_vehiculos = perfil_vehiculos.sort_values("SCORE_0_100", ascending=False).reset_index(drop=True)


In [77]:
#CREAR TABLA
# Seleccionar columnas relevantes
# ----------------------------
tabla = perfil_vehiculos[[
    "veh_canon",
    "NUM_ACCIDENTES",
    "SCORE_PROMEDIO_ACC",
    "PARQUE_TOTAL",
    "FACTOR_OBSOLESCENCIA",
    "SCORE_0_100"
]].copy()

tabla = tabla.rename(columns={
    "veh_canon": "Tipo de Vehículo",
    "NUM_ACCIDENTES": "N° Accidentes",
    "SCORE_PROMEDIO_ACC": "Severidad Promedio",
    "PARQUE_TOTAL": "Parque Automotor",
    "FACTOR_OBSOLESCENCIA": "Obsolescencia Promedio",
    "SCORE_0_100": "Índice de Riesgo (0–100)"
})

# --- CONVERSIÓN A FLOAT PARA EVITAR NaN VISUAL ---
num_cols = [
    "N° Accidentes",
    "Severidad Promedio",
    "Parque Automotor",
    "Obsolescencia Promedio",
    "Índice de Riesgo (0–100)"
]
tabla[num_cols] = tabla[num_cols].astype(float)

# ----------------------------
# Estilo: gradientes rojos
# ----------------------------
styled = (
    tabla.style
        .background_gradient(subset=["Severidad Promedio"], cmap="Reds")
        .background_gradient(subset=["Índice de Riesgo (0–100)"], cmap="Reds")
        .set_properties(**{
            'text-align': 'center',
            'font-size': '13px'
        })
        .hide(axis="index")
        .set_table_styles([
            {
                'selector': 'th.col_heading',
                'props': [
                    ('text-align', 'center'),
                    ('background-color', '#e6e6e6'),
                    ('color', 'black'),
                    ('font-weight', 'bold'),
                    ('font-size', '14px'),
                    ('border-bottom', '1px solid #999')
                ]
            },
            {
                'selector': 'td',
                'props': [
                    ('border', '1px solid #ccc')
                ]
            }
        ]).format({
            "N° Accidentes": "{:.0f}",
            "Severidad Promedio": "{:.2f}",
            "Parque Automotor": "{:.0f}",
            "Obsolescencia Promedio": "{:.2f}",
            "Índice de Riesgo (0–100)": "{:.2f}"
        })
)

styled


Tipo de Vehículo,N° Accidentes,Severidad Promedio,Parque Automotor,Obsolescencia Promedio,Índice de Riesgo (0–100)
MOTO,603,2.42,9107,0.65,100.00
BUS,18,2.16,292,0.85,48.25
CAMION,26,1.97,1081,0.69,30.88
CAMIONETA,38,1.88,1357,0.70,26.62
AUTOMOVIL,139,1.73,3965,0.71,26.41
VOLQUETA,4,1.67,134,0.85,14.33
BICICLETA,25,2.10,0,0.00,9.37
TRACTOCAMION,17,1.94,1,0.00,0.00


# CONSTRUCCIÓN HEÁGONOS Y MAPA

##01_generar_hexagonos_y_perfil

In [78]:
# 01_generar_hexagonos_y_perfil.py

RES = 9   # resolucion H3 (puedes cambiar a 8/9 según preferencia)

# ---------- Cargar datos (ajusta rutas) ----------
df_acc = pd.read_csv(ruta_acc, encoding="utf-8")
puntos_criticos = pd.read_csv(ruta_puntos, encoding="utf-8")

# ---------- normalizador simple ----------
def normalizar(txt):
    if pd.isna(txt): return ""
    return unicodedata.normalize("NFKD", str(txt)).encode("ASCII","ignore").decode().upper().strip()

# ---------- Asegurar coordenadas numéricas en puntos críticos ----------
# Algunos CSV traían comas en los números: "4,525,174" -> "4.525174"
def fix_coord(x):
    if pd.isna(x): return np.nan
    s = str(x).replace(",", ".")
    try:
        return float(s)
    except:
        # intentar quitar espacios y puntos extras
        s2 = s.replace(" ", "").replace("..", ".")
        try:
            return float(s2)
        except:
            return np.nan

puntos_criticos["Latitud"]  = puntos_criticos["Latitud"].apply(fix_coord)
puntos_criticos["Longitud"] = puntos_criticos["Longitud"].apply(fix_coord)

# ---------- Crear hex para accidentes ----------
df_acc["hex9"] = df_acc.apply(lambda r: h3.latlng_to_cell(float(r["Latitud"]), float(r["Longitud"]), RES), axis=1)

# ---------- Crear hex para puntos criticos ----------
puntos_criticos["hex9"] = puntos_criticos.apply(lambda r: h3.latlng_to_cell(float(r["Latitud"]), float(r["Longitud"]), RES), axis=1)

# ---------- Helper: obtener rango horario 2h ----------
def hour_from_time(s):
    # espera 'HH:MM:SS' o 'H:MM' etc.
    try:
        h = int(str(s).split(":")[0])
        return h
    except:
        return np.nan

df_acc["hora_int"] = df_acc["hora_siniestro"].apply(hour_from_time)
df_acc["hora_rango_2h"] = df_acc["hora_int"].apply(lambda h: f"{int(h//2*2):02d}-{int(h//2*2+2):02d}h" if not np.isnan(h) else "N/A")

# ---------- Agregados por hexágono ----------
agg_funcs = {
    "Id": ("count"),
    "cantidad_hombres": "sum",
    "cantidad_mujeres": "sum",
    "cantidad_involucrados": "sum",
    "SCORE_ACCIDENTE": "sum"  # si SCORE_ACCIDENTE no existe, coméntalo o crea uno antes
}

# asegurar columnas usadas (si SCORE_ACCIDENTE no existe, crear con 0)
if "SCORE_ACCIDENTE" not in df_acc.columns:
    df_acc["SCORE_ACCIDENTE"] = 0.0

# agrupar y luego calcular frecuencias y modos
grouped = df_acc.groupby("hex9")

rows = []
for hex_id, group in grouped:
    total_acc = len(group)
    sum_h = int(group["cantidad_hombres"].sum(min_count=1) if "cantidad_hombres" in group else 0)
    sum_m = int(group["cantidad_mujeres"].sum(min_count=1) if "cantidad_mujeres" in group else 0)
    sum_invol = int(group["cantidad_involucrados"].sum(min_count=1) if "cantidad_involucrados" in group else 0)
    score_sum = float(group["SCORE_ACCIDENTE"].sum())

    # día más frecuente
    dia_pred = group["day_name"].mode().iloc[0] if not group["day_name"].mode().empty else "N/A"
    # hora rango más frecuente
    hora_pred = group["hora_rango_2h"].mode().iloc[0] if not group["hora_rango_2h"].mode().empty else "N/A"
    # tipo accidente más frecuente
    tipo_acc_pred = group["tipo_accidente"].mode().iloc[0] if not group["tipo_accidente"].mode().empty else "N/A"
    # top 2 vehiculos (expande / separa por '/' o '-' en los strings)
    # separar vehículos compuestos por delimitadores comunes
    vehs = []
    for v in group["tipo_vehiculo"].astype(str).fillna("N/A"):
        # normalizar separadores
        for sep in ["/","-","–"," "]:
            v = v.replace(sep, "-")
        parts = [p.strip() for p in v.split("-") if p.strip()]
        vehs.extend(parts)
    veh_counts = Counter([normalizar(v) for v in vehs if v and v != "N/A"])
    top_vehs = [k for k,_ in veh_counts.most_common(2)]

    # categoria_edad predominante
    cat_edad = group["categoria_edad"].mode().iloc[0] if not group["categoria_edad"].mode().empty else "N/A"

    # puntos críticos dentro de ese hex
    pcs = puntos_criticos[puntos_criticos["hex9"] == hex_id]
    lista_puntos = []
    for _, pr in pcs.iterrows():
        lista_puntos.append({
            "Id": pr.get("Id"),
            "Direccion": pr.get("Direccion"),
            "Afectacion": pr.get("Afectacion"),
            "Categoria": pr.get("Categoria")
        })

    # centro latlon del hex
    lat_cent, lon_cent = h3.cell_to_latlng(hex_id)

    rows.append({
        "hex9": hex_id,
        "lat_cent": lat_cent,
        "lon_cent": lon_cent,
        "TOTAL_ACCIDENTES": total_acc,
        "SUM_HOMBRES": sum_h,
        "SUM_MUJERES": sum_m,
        "SUM_INVOLUCRADOS": sum_invol,
        "SCORE_TOTAL": score_sum,
        "DIA_MAS_FRECUENTE": dia_pred,
        "HORA_RANGO_MAS_FREC": hora_pred,
        "TIPO_ACC_MAS_FREC": tipo_acc_pred,
        "TOP_VEHICULOS": top_vehs,
        "CATEGORIA_EDAD_MAS_COMUN": cat_edad,
        "PUNTOS_CRITICOS": lista_puntos
    })

perfil_hex = pd.DataFrame(rows)

# Guardar
perfil_hex.to_csv("/content/perfil_hex9.csv", index=False)
print("✅ perfil_hex9 generado y guardado: /content/perfil_hex9.csv")

✅ perfil_hex9 generado y guardado: /content/perfil_hex9.csv


##02_calcular_score_por_hexagono

In [79]:
# 02_calcular_score_por_hexagono.py

# Cargar perfil_hex (generado anteriormente)
perfil_hex = pd.read_csv("/content/perfil_hex9.csv", converters={"PUNTOS_CRITICOS": ast.literal_eval, "TOP_VEHICULOS": ast.literal_eval})

# si tienes riesgo_obsolescencia (del cálculo del parque) lo cargas, sino se asume fallback
# riesgo_obsolescencia tiene columnas: TIPO_VEHICULO, FACTOR_OBSOLESCENCIA (valores 0..1)
try:
    riesgo_obsolescencia = pd.read_csv("/content/riesgo_obsolescencia.csv")  # si la guardaste antes
    riesgo_obsolescencia["norm"] = riesgo_obsolescencia["TIPO_VEHICULO"].astype(str).str.upper().str.replace(" ", "")
    dict_obsol = dict(zip(riesgo_obsolescencia["norm"], riesgo_obsolescencia["FACTOR_OBSOLESCENCIA"]))
except Exception:
    dict_obsol = {}

def obsol_promedio_para_vehs(vehs):
    # vehs: lista de strings normalizados/from perfil_hex TOP_VEHICULOS
    if not isinstance(vehs, list) or len(vehs)==0:
        return 0.0
    vals = []
    for v in vehs:
        key = str(v).upper().replace(" ", "")
        vals.append(dict_obsol.get(key, np.nan))
    vals = [v for v in vals if not np.isnan(v)]
    if len(vals)==0:
        return np.nan
    return float(np.mean(vals))

# Calcular obsolescencia estimada por hex (basado en top vehicles)
perfil_hex["FACTOR_OBSOLESCENCIA_EST"] = perfil_hex["TOP_VEHICULOS"].apply(obsol_promedio_para_vehs)

# Fallback: si NaN (p. ej. bicicletas) dejar 0.0 para que no penalice
perfil_hex["FACTOR_OBSOLESCENCIA_EST"] = perfil_hex["FACTOR_OBSOLESCENCIA_EST"].fillna(0.0)

# Normalizar componentes
max_acc = perfil_hex["TOTAL_ACCIDENTES"].replace(0, np.nan).max()
max_score_sum = perfil_hex["SCORE_TOTAL"].replace(0, np.nan).max()

perfil_hex["FREQ_NORM"] = perfil_hex["TOTAL_ACCIDENTES"] / (max_acc if pd.notna(max_acc) else 1)
perfil_hex["SCORE_SUM_NORM"] = perfil_hex["SCORE_TOTAL"] / (max_score_sum if pd.notna(max_score_sum) else 1)

# Composición del score por hex (ajustable)
# Ejemplo: 40% severidad (SCORE_SUM_NORM), 35% frecuencia (FREQ_NORM), 15% obsolescencia, 10% exposición simple
perfil_hex["SCORE_RAW"] = (
    perfil_hex["SCORE_SUM_NORM"] * 0.40 +
    perfil_hex["FREQ_NORM"] * 0.35 +
    perfil_hex["FACTOR_OBSOLESCENCIA_EST"] * 0.15 +
    0.10 * (perfil_hex["SUM_INVOLUCRADOS"] / (perfil_hex["SUM_INVOLUCRADOS"].max() if perfil_hex["SUM_INVOLUCRADOS"].max()>0 else 1))
)

# Normalizar 0-100
minv = perfil_hex["SCORE_RAW"].min()
maxv = perfil_hex["SCORE_RAW"].max()
if maxv == minv:
    perfil_hex["SCORE_0_100"] = 0
else:
    perfil_hex["SCORE_0_100"] = (perfil_hex["SCORE_RAW"] - minv) / (maxv - minv) * 100

# Guardar
perfil_hex.to_csv("/content/perfil_hex9_con_score.csv", index=False)
print("✅ perfil_hex9_con_score.csv guardado.")


✅ perfil_hex9_con_score.csv guardado.


##Generar recomendaciones

In [80]:
def generar_recomendacion_hex_v2(row):
    """
    row = fila de perfil_hex con columnas:
      - TOP_VEHICULOS (lista)
      - TIPO_ACC_MAS_FREC
      - DIA_MAS_FRECUENTE
      - HORA_RANGO_MAS_FREC
      - PUNTOS_CRITICOS (lista)
      - SCORE_0_100
    """

    recomendaciones = []

    # ------------------------------
    # Vehículos predominantes
    # ------------------------------
    vehs = row.get("TOP_VEHICULOS", [])
    vehs_norm = [str(v).upper() for v in vehs]

    if "MOTO" in vehs_norm:
        recomendaciones.append(
            "Aumentar controles de velocidad y campañas de visibilidad para motociclistas."
        )

    if "AUTOMOVIL" in vehs_norm:
        recomendaciones.append(
            "Verificar señalización horizontal y vertical en intersecciones críticas."
        )

    if any(v in vehs_norm for v in ["TRACTOCAMION", "CAMION", "VOLQUETA", "BUS"]):
        recomendaciones.append(
            "Inspeccionar radios de giro, desgaste del pavimento y maniobrabilidad en esquinas."
        )

    if "BICICLETA" in vehs_norm:
        recomendaciones.append(
            "Asegurar continuidad de ciclorrutas, buena iluminación y ausencia de obstáculos."
        )

    # ------------------------------
    # Tipo de accidente dominante
    # ------------------------------
    tipo = str(row.get("TIPO_ACC_MAS_FREC", "")).upper()

    if "CHOQUE" in tipo:
        recomendaciones.append("Revisar invasión de carril y giros peligrosos en la zona.")
    elif "ATROPELLO" in tipo:
        recomendaciones.append("Mejorar pasos peatonales e iluminación nocturna.")
    elif "CAIDA" in tipo or "CAÍDA" in tipo:
        recomendaciones.append("Corregir irregularidades del pavimento y tapas de alcantarilla.")
    elif "VOLCAMIENTO" in tipo:
        recomendaciones.append("Revisar peraltes, curvas cerradas y elementos de contención.")

    # ------------------------------
    # Horas y días
    # ------------------------------
    hora = str(row.get("HORA_RANGO_MAS_FREC", ""))
    dia  = str(row.get("DIA_MAS_FRECUENTE", ""))

    if "04" in hora or "05" in hora:
        recomendaciones.append("Incrementar control por fatiga o microsueño en la madrugada.")

    if dia.upper() in ["VIERNES", "SABADO", "SÁBADO"]:
        recomendaciones.append("Mayor vigilancia nocturna por posible consumo de alcohol.")

    # ------------------------------
    # Puntos críticos
    # ------------------------------
    puntos = row.get("PUNTOS_CRITICOS", [])
    if isinstance(puntos, list) and len(puntos) > 0:
        recomendaciones.append(
            "Este hexágono contiene puntos críticos oficiales: priorizar mantenimiento correctivo."
        )

    # ------------------------------
    # Score
    # ------------------------------
    score = float(row.get("SCORE_0_100", 0))

    if score >= 75:
        recomendaciones.append("Hexágono de prioridad ALTA: amerita intervención inmediata.")
    elif score >= 40:
        recomendaciones.append("Hexágono de riesgo MEDIO: recomienda reforzar medidas preventivas.")
    else:
        recomendaciones.append("Hexágono de riesgo BAJO: mantener seguimiento periódico.")

    # ------------------------------
    # Formato final
    # ------------------------------
    texto = "RECOMENDACIONES PARA ESTE HEXÁGONO:\n\n"
    for r in recomendaciones:
        texto += f"• {r}\n"

    return texto


##Generar pdfs por Hexágono

In [81]:
def generar_pdf_hexagono_v2(nombre_archivo, row, recomendacion):
    styles = getSampleStyleSheet()
    story = []

    story.append(Paragraph(f"Perfil de Riesgo – Hexágono #{int(row['HEX_ID'])}", styles["Title"]))
    story.append(Spacer(1, 12))

    story.append(Paragraph(f"<b>Score:</b> {row['SCORE_0_100']:.1f}/100", styles["Normal"]))
    story.append(Paragraph(f"<b>Total accidentes:</b> {int(row['TOTAL_ACCIDENTES'])}", styles["Normal"]))
    story.append(Paragraph(f"<b>Tipo accidente más frecuente:</b> {row['TIPO_ACC_MAS_FREC']}", styles["Normal"]))
    story.append(Paragraph(f"<b>Vehículos dominantes:</b> {', '.join(row['TOP_VEHICULOS'])}", styles["Normal"]))
    story.append(Paragraph(f"<b>Día más frecuente:</b> {row['DIA_MAS_FRECUENTE']}", styles["Normal"]))
    story.append(Paragraph(f"<b>Hora más frecuente:</b> {row['HORA_RANGO_MAS_FREC']}", styles["Normal"]))
    story.append(Spacer(1, 12))

    # Puntos críticos
    puntos = row["PUNTOS_CRITICOS"]
    if puntos:
        story.append(Paragraph("<b>Puntos críticos:</b>", styles["Heading3"]))
        for p in puntos:
            story.append(Paragraph(f"• {p['Direccion']}: {p['Afectacion']}", styles["Normal"]))
        story.append(Spacer(1, 12))

    # Recomendaciones
    story.append(Paragraph("<b>Recomendaciones:</b>", styles["Heading3"]))
    for line in recomendacion.split("\n"):
        story.append(Paragraph(line, styles["Normal"]))

    doc = SimpleDocTemplate(nombre_archivo, pagesize=letter)
    doc.build(story)
    return nombre_archivo

## Generar mapa interactivo

In [82]:
# 03_mapa_hex_profesional_mejorado.py

ruta_perfil_hex = "/content/perfil_hex9_con_score.csv"

# ----------------------------
# Cargar datos
# ----------------------------
perfil_hex = pd.read_csv(ruta_perfil_hex, converters={
    "PUNTOS_CRITICOS": lambda x: ast.literal_eval(x) if pd.notna(x) else [],
    "TOP_VEHICULOS": lambda x: ast.literal_eval(x) if pd.notna(x) else []
})
# The following lines are removed as df_acc and puntos_criticos are already in memory
# df_acc = pd.read_csv(ruta_acc)
# puntos_criticos = pd.read_csv(ruta_puntos)

# ----------------------------
# Normalizar / arreglar coords puntos_criticos
# ----------------------------
def fix_coord(x):
    if pd.isna(x): return None
    s = str(x).replace(',', '.').strip()
    try:
        return float(s)
    except:
        try:
            return float(s.replace(" ", ""))
        except:
            return None

puntos_criticos["Latitud"]  = puntos_criticos["Latitud"].apply(fix_coord)
puntos_criticos["Longitud"] = puntos_criticos["Longitud"].apply(fix_coord)

# ----------------------------
# Validaciones simples
# ----------------------------
# Asegurar columnas necesarias en perfil_hex:
expected_cols = ["hex9","lat_cent","lon_cent","TOTAL_ACCIDENTES","SUM_HOMBRES","SUM_MUJERES",
                 "SUM_INVOLUCRADOS","DIA_MAS_FRECUENTE","HORA_RANGO_MAS_FREC","TIPO_ACC_MAS_FREC",
                 "TOP_VEHICULOS","CATEGORIA_EDAD_MAS_COMUN","SCORE_0_100","PUNTOS_CRITICOS"]
for c in expected_cols:
    if c not in perfil_hex.columns:
        raise ValueError(f"Columna esperada no encontrada en perfil_hex: {c}")

# Reset index y generar ID secuencial legible
perfil_hex = perfil_hex.reset_index(drop=True)
perfil_hex["HEX_ID"] = perfil_hex.index + 1

# ----------------------------
# Util: boundary H3 -> list(lat,lon)
# ----------------------------
def hex_boundary_latlon(h):
    # h3.cell_to_boundary devuelve lista de (lat,lon) - sin argumentos extra
    b = h3.cell_to_boundary(h)
    # garantizar que esté en formato (lat, lon)
    return [(float(lat), float(lon)) for lat, lon in b]

# ----------------------------
# Crear mapa
# ----------------------------
centro = [perfil_hex["lat_cent"].mean(), perfil_hex["lon_cent"].mean()]
m = folium.Map(location=centro, zoom_start=13, tiles="CartoDB positron")

# ----------------------------
# MarkerCluster: Accidentes (capa)
# ----------------------------
mc_group = folium.FeatureGroup(name="Accidentes (cluster)", show=True)
mc = MarkerCluster().add_to(mc_group)
mc_group.add_to(m)

for _, r in df_acc.iterrows():
    try:
        lat = float(r["Latitud"])
        lon = float(r["Longitud"])
    except:
        continue
    popup_html = f"""
    <div style="font-family:Arial; font-size:12px; line-height:1.2; max-width:260px">
      <b>ID:</b> {r.get('Id','')}<br>
      <b>Tipo veh:</b> {r.get('tipo_vehiculo','')}<br>
      <b>Gravedad:</b> {r.get('gravedad','')}<br>
      <b>Tipo acc:</b> {r.get('tipo_accidente','')}<br>
      <b>Hora:</b> {r.get('hora_siniestro','')}
    </div>
    """
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        color="#196F9A",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(mc)

# ----------------------------
# Capa principal de hexágonos (un solo botón para todos)
# ----------------------------
hex_layer_parent = folium.FeatureGroup(name="Hexágonos (Todos)", show=True)
hex_layer_parent.add_to(m)

# Subcapas derivadas (FeatureGroupSubGroup) que comparten el mismo control
hex_pc_layer = FeatureGroupSubGroup(hex_layer_parent, "Hexágonos con Puntos Críticos")
hex_top10_layer = FeatureGroupSubGroup(hex_layer_parent, "Top 10 Hexágonos (más críticos)")
hex_filter_layer = FeatureGroupSubGroup(hex_layer_parent, "Hexágonos Score >= Umbral")

# Añadir subcapas al mapa (importante para que aparezcan en LayerControl)
hex_pc_layer.add_to(m)
hex_top10_layer.add_to(m)
hex_filter_layer.add_to(m)

# ----------------------------
# Colores por score y top10
# ----------------------------
min_s = perfil_hex["SCORE_0_100"].min()
max_s = perfil_hex["SCORE_0_100"].max()

def color_from_score(s):
    if pd.isna(s): return "#999999"
    r = (s - min_s) / (max_s - min_s) if max_s > min_s else 0
    if r > 0.66:
        return "#b10026"   # rojo
    elif r > 0.33:
        return "#f76824"   # naranja
    else:
        return "#ffd92f"   # amarillo

# Top10 set y umbral configurable
top10_hex_set = set(perfil_hex.sort_values("SCORE_0_100", ascending=False).head(10)["hex9"].tolist())
SCORE_THRESHOLD = 50.0  # puedes ajustar: hexágonos con score >= este valor aparecerán en la capa de filtro

# ----------------------------
# Dibujar hexágonos y construir popups
# ----------------------------
map_js_name = m.get_name()  # nombre de la variable JS usada en fitBounds

for _, row in perfil_hex.iterrows():
    h = row["hex9"]
    try:
        boundary = hex_boundary_latlon(h)  # lista de (lat,lon)
    except Exception as e:
        continue

    # construir HTML de puntos críticos (lista)
    puntos = row["PUNTOS_CRITICOS"]
    afectaciones_html = ""
    if isinstance(puntos, list) and len(puntos) > 0:
        afectaciones_html = "<ul style='margin:4px 0 4px 18px;padding:0'>"
        for p in puntos:
            # p puede ser dict {'Id':..,'Direccion':..,'Afectacion':..,'Categoria':..}
            if isinstance(p, dict):
                dirp = p.get("Direccion","")
                afect = p.get("Afectacion","")
                afectaciones_html += f"<li><b>{dirp}</b>: {afect}</li>"
            else:
                afectaciones_html += f"<li>{str(p)}</li>"
        afectaciones_html += "</ul>"
    else:
        afectaciones_html = "<i>Ninguno</i>"

    # TOP VEHÍCULOS (asegurar string)
    topveh = row["TOP_VEHICULOS"]
    if isinstance(topveh, list):
        topveh_txt = ", ".join(topveh) if topveh else "N/A"
    else:
        topveh_txt = str(topveh)

    # GENERAR RECOMENDACIÓN
    recomendacion = generar_recomendacion_hex_v2(row)

    # Generar el PDF (archivo temporal)
    nombre_pdf = f"hex_{int(row['HEX_ID'])}.pdf"
    generar_pdf_hexagono_v2(nombre_pdf, row, recomendacion)

    # Convertir PDF a base64
    encoded_pdf = base64.b64encode(open(nombre_pdf, "rb").read()).decode("utf-8")

    boton_reco = f"""
    <button onclick="alert(`{recomendacion.replace('`','')}`)"
      style="padding:6px 8px; background:#1F75FE; color:white; border:none; border-radius:4px; margin-top:6px;">
    Ver Recomendación
    </button>
    """

    boton_pdf = f"""
    <a download="{nombre_pdf}" href="data:application/pdf;base64,{encoded_pdf}">
       <button style="padding:6px 8px; background:#4CAF50; color:white; border:none; border-radius:4px; margin-top:6px;">
       Descargar PDF
    </button>
    </a>
    """

    popup_html = f"""
    <div style="font-family:Arial; font-size:13px; line-height:1.25; max-width:320px">
      <h4 style="margin:0 0 6px 0;">Hexágono #{int(row['HEX_ID'])} — ⭐ {row['SCORE_0_100']:.1f}/100</h4>
      <b>📊 Total accidentes:</b> {int(row['TOTAL_ACCIDENTES'])} —
      <b>👥 Involucrados:</b> {int(row['SUM_INVOLUCRADOS'])}<br>
      <b>👨 Hombres:</b> {int(row['SUM_HOMBRES'])} — <b>👩 Mujeres:</b> {int(row['SUM_MUJERES'])}<br>
      <b>⏱ Día más frecuente:</b> {row['DIA_MAS_FRECUENTE']} — <b>🕒 Hora:</b> {row['HORA_RANGO_MAS_FREC']}<br>
      <b>⚠️ Tipo + frecuente:</b> {row['TIPO_ACC_MAS_FREC']}<br>
      <b>🚗 Vehículos top:</b> {topveh_txt}<br>
      <b>👥 Edad más común:</b> {row['CATEGORIA_EDAD_MAS_COMUN']}<br><br>

    <b>🏷 Puntos críticos dentro del hex:</b>{afectaciones_html}

    <hr style="margin:6px 0">

    <button onclick="window['{map_js_name}'].fitBounds({json.dumps(boundary)})"
     style="padding:6px 8px; background:#196F9A; color:white; border:none; border-radius:4px">
     Zoom al hexágono
    </button>
    <br><br>

    {boton_reco}
    <br>
    {boton_pdf}
    </div>
    """

    # construir GeoJson polygon (note: folium espera coords lon,lat inside GeoJSON)
    polygon_geojson = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[ [lon, lat] for lat, lon in boundary ]]
        },
        "properties": {"hex9": h, "HEX_ID": int(row['HEX_ID']), "SCORE": row["SCORE_0_100"]}
    }

    # estilo base
    style_fn = lambda feat, row=row: {
        "color": color_from_score(row["SCORE_0_100"]),
        "weight": 2.2,
        "opacity": 0.95,
        "fillOpacity": 0.04
    }

    gj = folium.GeoJson(polygon_geojson, style_function=style_fn)
    gj.add_child(folium.Popup(popup_html, max_width=420))

    # Añadir a capa principal
    gj.add_to(hex_layer_parent)

    # Si tiene puntos críticos: clonar/añadir con relleno algo mayor en subcapa
    if isinstance(puntos, list) and len(puntos) > 0:
        style_pc = lambda feat, row=row: {
            "color": color_from_score(row["SCORE_0_100"]),
            "weight": 2.8,
            "opacity": 0.98,
            "fillOpacity": 0.12
        }
        gj_pc = folium.GeoJson(polygon_geojson, style_function=style_pc)
        gj_pc.add_child(folium.Popup(popup_html, max_width=420))
        gj_pc.add_to(hex_pc_layer)

    # Si pertenece al top10 → capa Top10 con relleno intenso
    if h in top10_hex_set:
        style_top = lambda feat, row=row: {
            "color": color_from_score(row["SCORE_0_100"]),
            "weight": 3.0,
            "opacity": 1.0,
            "fillOpacity": 0.28
        }
        gj_top = folium.GeoJson(polygon_geojson, style_function=style_top)
        gj_top.add_child(folium.Popup(popup_html, max_width=420))
        gj_top.add_to(hex_top10_layer)

    # Si supera el umbral SCORE_THRESHOLD -> capa filtro
    try:
        if float(row["SCORE_0_100"]) >= SCORE_THRESHOLD:
            style_thr = lambda feat, row=row: {
                "color": color_from_score(row["SCORE_0_100"]),
                "weight": 2.6,
                "opacity": 0.95,
                "fillOpacity": 0.18
            }
            gj_thr = folium.GeoJson(polygon_geojson, style_function=style_thr)
            gj_thr.add_child(folium.Popup(popup_html, max_width=420))
            gj_thr.add_to(hex_filter_layer)
    except:
        pass

# ----------------------------
# Añadir marcadores de puntos críticos (estrella)
# ----------------------------
pc_group = folium.FeatureGroup(name="Puntos Críticos", show=True)
pc_group.add_to(m)

for _, p in puntos_criticos.iterrows():
    lat = p["Latitud"]; lon = p["Longitud"]
    if pd.isna(lat) or pd.isna(lon): continue
    popup = folium.Popup(html=f"<b>📍 {p.get('Direccion','')}</b><br><b>Afectación:</b> {p.get('Afectacion','')}<br><b>Categoria:</b> {p.get('Categoria','')}", max_width=300)
    folium.Marker([lat, lon], icon=folium.Icon(icon="star", prefix="fa", color="darkred"), popup=popup).add_to(pc_group)

# ----------------------------
# Controles de capas (mantener filtros)
# ----------------------------
folium.LayerControl(collapsed=False).add_to(m)

# Guardar
OUT = "mapa_hex_profesional_mejorado.html"
m.save(OUT)
print("✅ Mapa guardado en:", OUT)

✅ Mapa guardado en: mapa_hex_profesional_mejorado.html
